In [28]:
# paibox 和 spikingjelly 的推理结果比较，输出为 csv 文件
# 需要修改数据集路径DVS128GESTURE_DATA_DIR
import torch
import numpy as np
import paibox as pb
import csv

def b(c):
    return np.random.randint(1,127,size=(c,))
def t():
    return np.random.randint(1,127,size=(1,))
def k(out_channels,in_channels,kernel_h=5,kernel_w=5):
    return np.random.randint(1,127,size=(out_channels,in_channels,kernel_h,kernel_w))
def w(in_channels,out_channels):
    return np.random.randint(1,127,size=(in_channels,out_channels))

# PAIBox网络定义
class Conv2d_Net(pb.Network):
    def __init__(self, channels, param_dict):
        super().__init__()

        self.i0 = pb.InputProj(input=None, shape_out=(1, 86, 65))
        self.n0 = pb.LIF((1, 42, 32), threshold=t(), reset_v=0, tick_wait_start=1) # convpool7x7p2s2
        self.conv2d_0 = pb.Conv2d(self.i0, self.n0, kernel=k(1,1,7,7), padding=2, stride=2)

        # self.n1_0 = pb.LIF((2, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # convpool7x7p3s2
        # self.conv2d_1_0 = pb.Conv2d(self.n0, self.n1_0, kernel=k(2,1,7,7), padding=3, stride=2)

        # self.n1_1 = pb.LIF((2, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # convpool7x7p3s2
        # self.conv2d_1_1 = pb.Conv2d(self.n0, self.n1_1, kernel=k(2,1,7,7), padding=3, stride=2)

        # self.n2_0 = pb.LIF((2, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # conv5x5
        # self.conv2d_2_0 = pb.FullConn(self.n1_0, self.n2_0, conn_type=pb.SynConnType.All2All, weights=w(2*21*16,2*21*16))
        # # self.conv2d_2_0_ = pb.FullConn(self.n1_1, self.n2_0, conn_type=pb.SynConnType.All2All, weights=w(2*21*16,2*21*16))

        # self.n2_1 = pb.LIF((2, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # conv5x5
        # self.conv2d_2_1 = pb.FullConn(self.n1_1, self.n2_1, conn_type=pb.SynConnType.All2All, weights=w(2*21*16,2*21*16))
        # # self.conv2d_2_1_ = pb.FullConn(self.n1_0, self.n2_1, conn_type=pb.SynConnType.All2All, weights=w(2*21*16,2*21*16))

        self.n1 = pb.LIF((2, 21, 16), threshold=t(), reset_v=0, tick_wait_start=2) # convpool7x7p3s2
        self.conv2d_1 = pb.Conv2d(self.n0, self.n1, kernel=k(2,1,7,7), padding=3, stride=2)

        self.n10 = pb.LIF(512, threshold=t(), reset_v=0, tick_wait_start=2) # fc
        # self.fc_0_0 = pb.FullConn(self.n2_0, self.n10, conn_type=pb.SynConnType.All2All, weights=w(2*21*16,512))
        # self.fc_0_1 = pb.FullConn(self.n2_1, self.n10, conn_type=pb.SynConnType.All2All, weights=w(2*21*16,512))
        self.fc_0 = pb.FullConn(self.n1, self.n10, conn_type=pb.SynConnType.All2All, weights=w(2*21*16,512))

        self.n11 = pb.LIF(128, threshold=t(), reset_v=0, tick_wait_start=3) # fc
        self.fc_1 = pb.FullConn(self.n10, self.n11, conn_type=pb.SynConnType.All2All, weights=w(512,128))

        self.n12 = pb.LIF(90, threshold=t(), reset_v=0, tick_wait_start=4) # fc
        self.fc_2 = pb.FullConn(self.n11, self.n12, conn_type=pb.SynConnType.All2All, weights=w(128,90))

        self.probe1 = pb.Probe(self.n12, "spike")

C = 2

# PAIBox网络初始化
param_dict = {}
def getNetParam():
    timestep = 8
    layer_num = 9
    delay = layer_num - 1
    param_dict["timestep"] = timestep
    param_dict["layer_num"] = layer_num
    param_dict["delay"] = delay

    param_dict['conv.1.weight']=np.random.randint(1,127,size=(2,1,7,7))
    param_dict['conv.1.bias']=np.random.randint(1,127,size=(2))
    param_dict['conv.0.weight']=np.random.randint(1,127,size=(C*2,2,7,7))
    param_dict['conv.0.bias']=np.random.randint(1,127,size=(C*2))
    param_dict['conv.2.weight']=np.random.randint(1,127,size=(C*2,C*2,5,5))
    param_dict['conv.2.bias']=np.random.randint(1,127,size=(C*2))
    param_dict['conv.4.weight']=np.random.randint(1,127,size=(C*4,C*2,5,5))
    param_dict['conv.4.bias']=np.random.randint(1,127,size=(C*4))
    param_dict['conv.6.weight']=np.random.randint(1,127,size=(C*4,C*4,5,5))
    param_dict['conv.6.bias']=np.random.randint(1,127,size=(C*4))
    param_dict['conv.8.weight']=np.random.randint(1,127,size=(C*8,C*4,5,5))
    param_dict['conv.8.bias']=np.random.randint(1,127,size=(C*8))
    param_dict['conv.10.weight']=np.random.randint(1,127,size=(C*8,C*8,5,5))
    param_dict['conv.10.bias']=np.random.randint(1,127,size=(C*8))
    param_dict['fc.2.weight']=np.random.randint(1,127,size=(C*2*21*16,C*2*21*16)).T
    param_dict['fc.5.weight']=np.random.randint(1,127,size=(C*2*21*16,C*2*21*16)).T
    param_dict['fc.8.weight']=np.random.randint(1,127,size=(C*2*21*16,C*2*21*16)).T
    param_dict['fc.11.weight']=np.random.randint(1,127,size=(C*2*21*16,C*2*21*16)).T
    param_dict['fc.14.weight']=np.random.randint(1,127,size=(C*2*21*16,C*2*21*16)).T
    param_dict['fc.17.weight']=np.random.randint(1,127,size=(90,C*2*21*16)).T
    param_dict['conv.1.vthr']=10
    param_dict['conv.0.vthr']=10
    param_dict['conv.2.vthr']=20
    param_dict['conv.4.vthr']=30
    param_dict['conv.6.vthr']=40
    param_dict['conv.8.vthr']=50
    param_dict['conv.10.vthr']=60
    param_dict['fc.2.vthr']=70
    param_dict['fc.5.vthr']=80
    param_dict['fc.8.vthr']=90
    param_dict['fc.11.vthr']=100
    param_dict['fc.14.vthr']=110
    param_dict['fc.17.vthr']=120
getNetParam()

# PAIBox仿真器
pb_net = Conv2d_Net(C, param_dict)
sim = pb.Simulator(pb_net)


# test(test_num=50)
pb.BACKEND_CONFIG.target_chip_addr = (0, 0)

# print(pb_net.conv2d_0.name)
# print(type(pb_net.n0))
# print(pb_net.n0.shape_in)
# print(pb_net.n0.shape_out)
# print(type(pb_net.n0[0,:,:]))
# print(pb_net.n0[0,:,:].shape_in)
# print(pb_net.n0[0,:,:].shape_out)
# n1_0_1 = (pb_net.n11,pb_net.n11)
# print(type(n1_0_1))

c:\Users\oilgi\AppData\Local\Programs\Python\Python310\lib\site-packages\paibox\components\synapses\transforms.py:104: AutoOptimizationWarning: dtype of weight is optimized automatically, int32 -> int8.
  warnings.warn(


In [29]:
mapper = pb.Mapper()

mapper.build(pb_net)

# Core estimate only
graph_info = mapper.compile(
    weight_bit_optimization=True, grouping_optim_target="both", core_estimate_only=True
)

# #N of cores required
print("Core required:", graph_info["n_core_required"])

# Clear all the results
mapper.clear()

RoutingGroup: RoutingGroup_155 with 0 cores:
multicast axons: ['InputProj_15']
	CoreBlock_75 with 0 cores:
		LCN: 3
		Conv2d_39: InputProj_15 -> LIF_93

RoutingGroup: RoutingGroup_156 with 0 cores:
multicast axons: ['LIF_93']
	CoreBlock_76 with 0 cores:
		LCN: 1
		Conv2d_40: LIF_93 -> LIF_94

RoutingGroup: RoutingGroup_157 with 0 cores:
multicast axons: ['LIF_95']
	CoreBlock_77 with 0 cores:
		LCN: 0
		FullConn_67: LIF_95 -> LIF_96

RoutingGroup: RoutingGroup_158 with 0 cores:
multicast axons: ['LIF_94']
	CoreBlock_78 with 0 cores:
		LCN: 0
		FullConn_66: LIF_94 -> LIF_95

RoutingGroup: RoutingGroup_159 with 0 cores:
multicast axons: ['LIF_96']
	CoreBlock_79 with 0 cores:
		LCN: 0
		FullConn_68: LIF_96 -> LIF_97

Core required: 201


In [3]:
mapper = pb.Mapper()

mapper.build(pb_net)

graph_info = mapper.compile(
    weight_bit_optimization=True, grouping_optim_target="both"
)

# #N of cores required
print("Core required:", graph_info["n_core_required"])

Core required: 897


In [ ]:
mapper.export(
    write_to_file=True, fp="./debug", format="npy", export_core_params=False
)

# Clear all the results
mapper.clear()